# Loading Important Libraries

In [37]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import shutil
import cv2
import py360convert
import re
import folium
import plotly.express as px


# Image Preprocessing

In [18]:
# We do a first inspection, i.e. do the images have the same size, are we able to detect the black frames, etc.?
image_folder = 'C:/Users/Nutzer/Downloads/images(1)/images'

def inspect_images(image_root, num_images_per_folder=3):
    for subfolder in sorted(os.listdir(image_root)):
        subfolder_path = os.path.join(image_root, subfolder)
        if not os.path.isdir(subfolder_path):
            continue

        print(f"Inspecting images from folder: '{subfolder}'")
        images = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        print(f"  Total images: {len(images)}")

        # Inspect a few images per folder
        for img_name in images[:num_images_per_folder]:
            img_path = os.path.join(subfolder_path, img_name)
            with Image.open(img_path) as img:
                width, height = img.size
                np_img = np.array(img)
                img_mean_pixel_value = np.mean(np_img)

                # Check for black frame (right edge)
                right_edge = np_img[:, -1, :]
                right_edge_mean = np.mean(right_edge)

                print(f"    Image: {img_name}")
                print(f"       - Dimensions: {width}px (width) × {height}px (height)")
                print(f"       - Mean Pixel Value: {img_mean_pixel_value:.2f}")
                if right_edge_mean < 10:
                    print(f"       - Possible black frame detected (right edge mean: {right_edge_mean:.2f})")
                else:
                    print(f"       - No black frame detected (right edge mean: {right_edge_mean:.2f})")

if __name__ == "__main__":
    inspect_images(image_folder)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Nutzer/Downloads/images(1)/images'

Resizing and Cleaning: The images are really big, to make the processing faster we resize. Furthermore, we detected some black frames on the right and bottom side of some pictures. Those shall be removed.

In [ ]:
# Resize and remove the black frames, split the data and save the folders
original_dir = 'C:/Users/Nutzer/Downloads/images(1)/images'
processed_dir = 'processed_images_final'
resize_dim_final = (4096, 2048)    # Direct final resize

# Function to resize directly to final dimensions
def resize_final(img):
    return img.resize(resize_dim_final, Image.LANCZOS)

# Crop function remains same
def remove_black_borders(img, threshold=10):
    img_np = np.array(img)
    gray = np.mean(img_np, axis=2)
    mask = gray > threshold
    coords = np.argwhere(mask)
    
    if coords.size == 0:
        # Fully black or problematic image
        print("Image fully black after resizing. Saving without cropping.")
        return img  # Return original resized image without cropping
    
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    return img.crop((x0, y0, x1, y1))

# Processing loop (one resize step)
panorama_images, labels = [], []

for country in os.listdir(original_dir):
    country_path = os.path.join(original_dir, country)
    if not os.path.isdir(country_path):
        continue

    for img_name in os.listdir(country_path):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(country_path, img_name)

            img = Image.open(img_path)
            img_resized = resize_final(img)
            img_cropped = remove_black_borders(img_resized)

            # Save directly
            final_country_dir = os.path.join(processed_dir, country)
            os.makedirs(final_country_dir, exist_ok=True)
            final_img_path = os.path.join(final_country_dir, img_name)
            img_cropped.save(final_img_path, 'JPEG', quality=85)

            panorama_images.append(final_img_path)
            labels.append(country)

            print(f"Processed and saved: {country}/{img_name}")


Processed and saved: argentina/1741691006_-30.9927065_-68.8548332.jpg
Processed and saved: argentina/1741691007_-22.8471847_-65.2269743.jpg
Processed and saved: argentina/1741691009_-22.8081965_-65.8481546.jpg
Processed and saved: argentina/1741691010_-22.1328156_-65.743084.jpg
Processed and saved: argentina/1741691010_-27.8027889_-68.02298.jpg
Processed and saved: argentina/1741691011_-22.8455259_-66.3767022.jpg
Processed and saved: argentina/1741691011_-38.7294256_-70.8654145.jpg
Processed and saved: argentina/1741691013_-22.5940006_-65.8479498.jpg
Processed and saved: argentina/1741691020_-24.1420579_-65.4671979.jpg
Processed and saved: argentina/1741691022_-24.1510441_-65.4733955.jpg
Processed and saved: argentina/1741691087_-38.7232966_-70.8626231.jpg
Processed and saved: argentina/1741691093_-22.8745366_-65.2455576.jpg
Processed and saved: argentina/1741691094_-22.7924147_-65.2197581.jpg
Processed and saved: argentina/1741691095_-22.7437566_-65.9205854.jpg
Processed and saved: ar

ValueError: zero-size array to reduction operation minimum which has no identity

Image Transformation: The images are equirectangular. Thus, we can extract 4 perspectives from one picture. We do that using the cv2 and py360convert libraries. 

In [28]:
input_dir = "processed_images_final"
# Output directory for perspective images
output_dir = "processed_images_final_perspectives"
os.makedirs(output_dir, exist_ok=True)

# Output size for perspective images (width, height)
img_width, img_height = 1024, 1024
# Field-of-view: pass as a tuple (horizontal_fov, vertical_fov)
fov_deg = (90, 90)
# Number of perspective views to extract per panorama
num_views = 4

def extract_views(equi_img, num_views, fov_deg, out_hw):
    """
    Extract perspective views from an equirectangular image.
    The function generates `num_views` views using equally spaced yaw angles.
    """
    yaw_angles = np.linspace(-180, 180, num_views, endpoint=False)
    views = []
    for yaw in yaw_angles:
        perspective = py360convert.e2p(
            equi_img, 
            fov_deg=fov_deg, 
            u_deg=yaw, 
            v_deg=0, 
            out_hw=out_hw, 
            mode='bilinear'
        )
        views.append(perspective)
    return views

# Process each country folder
for country in os.listdir(input_dir):
    country_path = os.path.join(input_dir, country)
    if not os.path.isdir(country_path):
        continue

    # Create corresponding output folder for the country
    output_country_path = os.path.join(output_dir, country)
    os.makedirs(output_country_path, exist_ok=True)

    # Process each image file in the country folder
    for img_name in os.listdir(country_path):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        input_img_path = os.path.join(country_path, img_name)
        equi_img = cv2.imread(input_img_path)
        if equi_img is None:
            print(f"Warning: Could not read image {input_img_path}")
            continue
        # Convert BGR to RGB for processing
        equi_img = cv2.cvtColor(equi_img, cv2.COLOR_BGR2RGB)

        # Extract perspective views from the equirectangular image
        views = extract_views(equi_img, num_views, fov_deg, (img_height, img_width))

        # Save each perspective view with a filename that starts with the original name
        for i, view in enumerate(views):
            base_name, ext = os.path.splitext(img_name)
            new_filename = f"{base_name}_view_{i+1}.jpg"
            save_path = os.path.join(output_country_path, new_filename)
            # Convert RGB back to BGR for cv2.imwrite
            view_bgr = cv2.cvtColor(view, cv2.COLOR_RGB2BGR)
            cv2.imwrite(save_path, view_bgr)

        # (Exemplary output for one image)
        print(f"Processed {img_name} in {country} -> saved {num_views} perspective views.")

print("All images processed and saved in their respective country folders.")

Processed 1741691006_-30.9927065_-68.8548332.jpg in argentina -> saved 4 perspective views.
Processed 1741691007_-22.8471847_-65.2269743.jpg in argentina -> saved 4 perspective views.
Processed 1741691009_-22.8081965_-65.8481546.jpg in argentina -> saved 4 perspective views.
Processed 1741691010_-22.1328156_-65.743084.jpg in argentina -> saved 4 perspective views.
Processed 1741691010_-27.8027889_-68.02298.jpg in argentina -> saved 4 perspective views.
Processed 1741691011_-22.8455259_-66.3767022.jpg in argentina -> saved 4 perspective views.
Processed 1741691011_-38.7294256_-70.8654145.jpg in argentina -> saved 4 perspective views.
Processed 1741691013_-22.5940006_-65.8479498.jpg in argentina -> saved 4 perspective views.
Processed 1741691020_-24.1420579_-65.4671979.jpg in argentina -> saved 4 perspective views.
Processed 1741691022_-24.1510441_-65.4733955.jpg in argentina -> saved 4 perspective views.
Processed 1741691087_-38.7232966_-70.8626231.jpg in argentina -> saved 4 perspectiv

Splitting the Data

In [29]:
# Directories
input_dir = "processed_images_final_perspectives"
train_dir = "final_dataset/train"
test_dir = "final_dataset/test"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Set the test split ratio (e.g., 20% for test, 80% for training)
test_size = 0.2

# Process each country folder in the input directory
for country in os.listdir(input_dir):
    country_input_path = os.path.join(input_dir, country)
    if not os.path.isdir(country_input_path):
        continue
    
    # List all image files in the current country folder
    images = [f for f in os.listdir(country_input_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Split images into training and test sets
    train_imgs, test_imgs = train_test_split(images, test_size=test_size, random_state=42)
    
    # Create corresponding output folders for this country in train and test directories
    country_train_dir = os.path.join(train_dir, country)
    country_test_dir = os.path.join(test_dir, country)
    os.makedirs(country_train_dir, exist_ok=True)
    os.makedirs(country_test_dir, exist_ok=True)
    
    # Copy training images
    for img in train_imgs:
        src = os.path.join(country_input_path, img)
        dst = os.path.join(country_train_dir, img)
        shutil.copy(src, dst)
    
    # Copy test images
    for img in test_imgs:
        src = os.path.join(country_input_path, img)
        dst = os.path.join(country_test_dir, img)
        shutil.copy(src, dst)
    
    print(f"Processed folder '{country}': {len(train_imgs)} train, {len(test_imgs)} test images.")

print("Dataset successfully split into train and test sets.")


Processed folder 'argentina': 476 train, 120 test images.
Processed folder 'austria': 857 train, 215 test images.
Processed folder 'canada': 800 train, 200 test images.
Processed folder 'chile': 793 train, 199 test images.
Processed folder 'france': 704 train, 176 test images.
Processed folder 'iceland': 377 train, 95 test images.
Processed folder 'italy': 800 train, 200 test images.
Processed folder 'japan': 800 train, 200 test images.
Processed folder 'new_zealand': 640 train, 160 test images.
Processed folder 'norway': 736 train, 184 test images.
Processed folder 'peru': 508 train, 128 test images.
Processed folder 'switzerland': 976 train, 244 test images.
Dataset successfully split into train and test sets.


# Data Exploration

To get an overview, we explore the positions of the places portrayed via a map. 

In [ ]:
# Set the path to your directory containing the image files
root_directory = 'final_dataset/train'

# Regular expression pattern to match filenames like:
# 1741691006_-30.9927065_-68.8548332_view_1.jpg
pattern = re.compile(r'(\d+)_(-?\d+\.\d+)_(-?\d+\.\d+)_view_(\d+)(?:\.jpg)?$')

coords = []

# Walk through all subdirectories and files
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        match = pattern.match(filename)
        if match:
            timestamp, lat, lon, view = match.groups()
            coords.append((float(lat), float(lon)))
        else:
            # Optional: print filenames that don't match for debugging
            # print("No match for:", os.path.join(subdir, filename))
            pass

# Remove duplicate coordinates since each location may have multiple views
unique_coords = list(set(coords))

if not unique_coords:
    print("No coordinates extracted.")
else:
    # Create a DataFrame from the unique coordinates
    df = pd.DataFrame(unique_coords, columns=['lat', 'lon'])
    print("Extracted coordinates:")
    print(df.head())

    # Create an interactive map using Plotly Express
    fig = px.scatter_geo(
        df,
        lat='lat',
        lon='lon',
        title="Image Locations Map",
        projection="natural earth",
        hover_name="lat"  
    )

    # Save the interactive map to an HTML file
    fig.update_traces(marker=dict(color='red', size=5))
    fig.write_html("map.html")

Extracted coordinates:
         lat        lon
0  47.045577   9.075337
1 -11.539496 -76.279069
2  45.968461   7.114060
3 -33.347556 -70.361916
4 -22.847340 -65.227987


In [ ]:
# We detected some markers in the Indian Ocean which are suspicious. Investigate points where lat < -10 and lon between 50 and 100. 
df_suspicious = df[(df['lat'] < -10) & (df['lon'] > 50) & (df['lon'] < 100)]
print("Suspicious coordinates:")
print(df_suspicious)



Suspicious coordinates:
            lat        lon
738  -21.058772  55.368674
1014 -21.221322  55.680430
1785 -21.232792  55.667724


In [49]:
# Investigate suspicious coordinates (-> probably belong to France, e.g. La Reunion)

# List to store tuples of (latitude, longitude, full_file_path)
coords_with_file = []

# Walk through subdirectories
for subdir, dirs, files in os.walk(root_directory):
    for filename in files:
        match = pattern.match(filename)
        if match:
            timestamp, lat, lon, view = match.groups()
            full_path = os.path.join(subdir, filename)
            coords_with_file.append((float(lat), float(lon), full_path))

# Create dataframe
df = pd.DataFrame(coords_with_file, columns=['lat', 'lon', 'filepath'])

# Three suspicious coordinates:
suspicious_coords = [
    (-21.058772, 55.368674),
    (-21.221322, 55.680430),
    (-21.232792, 55.667724)
]

# Because of floating point precision issues, it's best to compare with a tolerance:
def is_close(row, coord, tol=1e-6):
    return (abs(row['lat'] - coord[0]) < tol) and (abs(row['lon'] - coord[1]) < tol)

# For each suspicious coordinate, filter and print the corresponding files:
for coord in suspicious_coords:
    filtered = df[df.apply(lambda row: is_close(row, coord), axis=1)]
    print("\nFor suspicious coordinate {}:".format(coord))
    if filtered.empty:
        print("  No files found for this coordinate.")
    else:
        print(filtered[['filepath']])



For suspicious coordinate (-21.058772, 55.368674):
                                               filepath
3276  final_dataset/train\france\1741690402_-21.0587...
3277  final_dataset/train\france\1741690402_-21.0587...
3278  final_dataset/train\france\1741690402_-21.0587...
3279  final_dataset/train\france\1741690402_-21.0587...

For suspicious coordinate (-21.221322, 55.68043):
                                               filepath
3454  final_dataset/train\france\1741696613_-21.2213...
3455  final_dataset/train\france\1741696613_-21.2213...

For suspicious coordinate (-21.232792, 55.667724):
                                               filepath
3032  final_dataset/train\france\1741647286_-21.2327...
3033  final_dataset/train\france\1741647286_-21.2327...
3034  final_dataset/train\france\1741647286_-21.2327...
3035  final_dataset/train\france\1741647286_-21.2327...


Note: Those pictures probably have to be treated as outliers. The vegetation might be very different to those of the rest of France and, more important, if we aim to predict coordinates, 10 images are 